Importing Libraries

In [1]:
import tensorflow as tf

Preprocessing

In [ ]:
training_set = tf.keras.utils.image_dataset_from_directory(directory='dataset/training_set', image_size=(64,64), batch_size = 32, label_mode='binary') #use label_mode = 'int' for multiple categories

test_set = tf.keras.utils.image_dataset_from_directory(directory='dataset/test_set', image_size=(64,64), batch_size=32, label_mode='binary')

data_augmentation_train = tf.keras.Sequential([
    tf.keras.layers.Rescaling(1./255),
    tf.keras.layers.RandomZoom(0.2),
    tf.keras.layers.RandomFlip('horizontal'),
    tf.keras.layers.RandomShear(0.2)
])

data_augmentation_test = tf.keras.Sequential([
    tf.keras.layers.Rescaling(1./255)
])

Found 8000 files belonging to 2 classes.
Found 2000 files belonging to 2 classes.


Building the CNN

True
